In [142]:
import torch.nn as nn
import torch
import torch.optim as optim
import numpy as np
import struct
import matplotlib.pyplot as plt

In [ ]:
train_image = open('train-images.idx3-ubyte', 'rb')
test_image = open('t10k-images.idx3-ubyte', 'rb')
train_label = open('train-labels.idx1-ubyte', 'rb')
test_label = open('t10k-labels.idx1-ubyte', 'rb')

In [ ]:
# 按照给定的格式进行解析，不知道干啥用的 尤其是这个>II 没太搞懂，而且他这块只弄标签
magic, n = struct.unpack('>II', train_label.read(8))

In [ ]:
# 原始数据的标签 从文本或二进制文件中的数据构造一个数组
# uint8类型的数据对象可以保证数组内的所有数值的大小都在 [0, 255]之间。 而图像数据中的每个像素点的值都是256个灰度级
# ndmin代表数组的维数
y_train_label = np.array(np.fromfile(train_label, dtype=np.uint8), ndmin=1)
print(y_train_label)
print(len(y_train_label))

In [ ]:
# 我的理解是10行60000列，每个的值是0.01
# 有一个小问题 这里是不是应该是0.001
y_train = np.ones((10, 60000)) * 0.01
print(y_train)
print(np.size(y_train,0)) # 行数
print(np.size(y_train,1)) # 列数

In [ ]:
# y_train现在是10行60000列
# 训练样本总共有60000个 相当于把每一个样本的那一列的label的那一行标注为1
for i in range(60000):
    y_train[y_train_label[i]][i] = 0.99
print(y_train)

In [ ]:
# 测试数据加载
magic_t, n_t = struct.unpack('>II',test_label.read(8))
# 测试的标签结果
y_test = np.fromfile(test_label,dtype=np.uint8).reshape(10000, 1)
print(y_test)

In [ ]:
# 在这里magic和第二个cell中的magic不冲突么？
# 还是这里unpack不太懂
magic, num, rows, cols = struct.unpack('>IIII', train_image.read(16))
print(magic,num,rows,cols)
# x_train的构成 60000行 784列
# 相当于60000个测试样例被一列一列展开了
x_train = np.fromfile(train_image, dtype=np.uint8).reshape(len(y_train_label), 784)
print(x_train)
print(np.size(x_train,0)) # 行数
print(np.size(x_train,1)) # 列数

In [ ]:
# 这个magic真的不知道在干嘛
magic_2, num_2, rows_2, cols_2 = struct.unpack('>IIII', test_image.read(16))
print(magic_2,num_2,rows_2,cols_2)
x_test = np.fromfile(test_image, dtype=np.uint8).reshape(len(y_test), 784)
print(x_test)
# 测试样例10000行784列
# 测试样例被按列展开了
print(np.size(x_test,0)) # 行数
print(np.size(x_test,1)) # 列数

In [ ]:
train_image.close()
train_label.close()
test_image.close()
test_label.close()

In [ ]:
class convolution_neural_network(nn.Module):
    def __init__(self):
        super(convolution_neural_network, self).__init__()

        # 定义卷积层
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0),  # 28x28x1-->24x24x6
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 12x12x6
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),  # 8x8x16
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 4x4x16
        )
        self.fc = nn.Sequential(
            nn.Linear(in_features=256, out_features=120),
            nn.Sigmoid(),
            nn.Linear(in_features=120, out_features=84),
            nn.Sigmoid(),
            nn.Linear(in_features=84, out_features=10),
        )

    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output